# Import

In [3]:
import pandas as pd
import numpy as np
from scipy.special import loggamma
import itertools

In [4]:
import random
import operator

In [5]:
def prob(M: np.matrix, order: [int]):
    p = 0.0
    for i in range(1, len(order)):
        sl = np.array(2 * M[:, order[i - 1]] + M[:, order[i]])
        ns = [sum(sl == j) for j in range(0, 4)]
        p -= loggamma(ns[0] + ns[1] + 1/2) + loggamma(ns[2] + ns[3] + 1/2)
        p += sum(map(lambda n: loggamma(n + 1/4), ns))
    return p

# Data

In [6]:
genes_matrix = np.array(pd.read_csv('genes_matrix.txt', header = None, delimiter= " "))

In [7]:
genes_matrix

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

# Genetic algorithm

In [48]:
def createSequence(genesList):
    route = random.sample(genesList, len(genesList))
    return route

def initialPopulation(popSize, genesList):
    population = []
    for i in range(0, popSize):
        population.append(createSequence(genesList))
    return population

def rankSequences(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = prob(genes_matrix, population[i])
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            gene1 = individual[swapped]
            gene2 = individual[swapWith]
            
            individual[swapped] = gene2
            individual[swapWith] = gene1
    return individual

def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankSequences(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial logProbability: " + str(rankSequences(pop)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        print("Current logProbability: " + str(rankSequences(pop)[0][1]))
    
    print("Final logProbability: " + str(rankSequences(pop)[0][1]))
    bestSequenceIndex = rankSequences(pop)[0][0]
    bestSequence = pop[bestSequenceIndex]
    return bestSequence

In [49]:
population = [i for i in range(20)]

In [50]:
geneticAlgorithm(population, popSize = 100, eliteSize = 20,
                 mutationRate = 0.01, generations = 500)

Initial logProbability: -972.9084231949925
Current logProbability: -962.7077774824165
Current logProbability: -962.7077774824165
Current logProbability: -950.8590346083398
Current logProbability: -950.8590346083398
Current logProbability: -933.1058554576643
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1948799953288
Current logProbability: -921.1267426711079
Current logProbability: -921.1267426711079
Current logProbability: -907.5566786449916
Current logProbability: -907.5566786449916
Current logProbability: -907.5566786449916
Current logProbability: -904.3642645002927
Current logProbability: -904.3642645002927
Current logProbability: -904.3642645002927
Current log

Current logProbability: -876.3272079214593
Current logProbability: -876.3272079214593
Current logProbability: -876.3272079214593
Current logProbability: -876.3272079214593
Current logProbability: -876.3272079214593
Current logProbability: -876.3272079214593
Current logProbability: -876.3272079214593
Current logProbability: -878.5875893756406
Current logProbability: -878.5875893756406
Current logProbability: -878.9245654857883
Current logProbability: -878.9245654857883
Current logProbability: -878.9245654857883
Current logProbability: -878.9245654857883
Current logProbability: -878.9245654857883
Current logProbability: -878.9245654857883
Current logProbability: -878.9245654857883
Current logProbability: -879.0372066825904
Current logProbability: -879.0372066825904
Current logProbability: -878.8602434514611
Current logProbability: -878.8602434514611
Current logProbability: -878.8602434514611
Current logProbability: -878.8602434514611
Current logProbability: -878.8602434514611
Current log

Current logProbability: -879.1714635618787
Current logProbability: -879.1714635618787
Current logProbability: -879.1714635618787
Current logProbability: -879.1714635618787
Current logProbability: -879.5961204498722
Current logProbability: -879.5961204498722
Current logProbability: -877.763509163802
Current logProbability: -877.763509163802
Current logProbability: -877.763509163802
Current logProbability: -879.5286730675009
Current logProbability: -879.5286730675009
Current logProbability: -879.5022855365967
Current logProbability: -879.5022855365967
Current logProbability: -879.5022855365967
Current logProbability: -879.3125980278355
Current logProbability: -879.5022855365967
Current logProbability: -879.5022855365967
Current logProbability: -879.5022855365967
Current logProbability: -879.5022855365967
Current logProbability: -879.5022855365967
Current logProbability: -879.9572645395572
Current logProbability: -878.6566641429253
Current logProbability: -878.6566641429253
Current logPro

[6, 7, 8, 9, 1, 19, 15, 16, 5, 14, 10, 2, 3, 4, 17, 18, 11, 12, 13, 0]

In [370]:
prob(genes_matrix, [6, 7, 8, 9, 1, 19, 15, 16, 5, 14, 10, 2, 3, 4, 17, 18, 11, 12, 13, 0])

-877.7898989039105

# Simulated annealing

In [213]:
import random
import math
from copy import deepcopy

In [379]:
def createSequence(genesList):
    route = random.sample(genesList, len(genesList))
    return route

def get_neighbor(ori_sequence):
    sequence = deepcopy(ori_sequence)
    swapped = int(random.random() * len(sequence))
    swapWith = int(random.random() * len(sequence))
    
    while swapped == swapWith:
        swapWith = int(random.random() * len(sequence))

    gene1 = sequence[swapped]
    gene2 = sequence[swapWith]

    sequence[swapped] = gene2
    sequence[swapWith] = gene1
    return sequence

def simulatedAnnealing(start, startTemp, endTemp, stepTemp, n_iter):
    currSequence = start
    currTemp = startTemp
    currProb = prob(genes_matrix, currSequence)
    
    while currTemp >= endTemp and currProb <= -1:
        for i in range(n_iter):
            currProb = prob(genes_matrix, currSequence)
            neighborSequence = get_neighbor(currSequence)
            neighborProb = prob(genes_matrix, neighborSequence)
            diff = neighborProb - currProb
            
            if diff >= 0:
                currSequence = neighborSequence
            else:
                if random.uniform(0, 1) < math.exp(diff / currTemp):
                    currSequence = neighborSequence
        currTemp -= stepTemp
        print("Current temperature: " + str(round(currTemp, 1)))
        print("Current logProbability: " + str(prob(genes_matrix, currSequence)))
    return currSequence, prob(genes_matrix, currSequence)

In [380]:
results = simulatedAnnealing(seq, 10, 1, 0.1, 500)

Current temperature: 9.9
Current logProbability: -966.9015590638008
Current temperature: 9.8
Current logProbability: -940.7017627057743
Current temperature: 9.7
Current logProbability: -949.5278652206293
Current temperature: 9.6
Current logProbability: -954.1573786387805
Current temperature: 9.5
Current logProbability: -937.2615065139667
Current temperature: 9.4
Current logProbability: -1014.4295463091036
Current temperature: 9.3
Current logProbability: -922.068109355871
Current temperature: 9.2
Current logProbability: -975.0589442104949
Current temperature: 9.1
Current logProbability: -993.1957525138007
Current temperature: 9.0
Current logProbability: -968.374567629786
Current temperature: 8.9
Current logProbability: -972.4480670113866
Current temperature: 8.8
Current logProbability: -968.4474718351058
Current temperature: 8.7
Current logProbability: -946.7859606794724
Current temperature: 8.6
Current logProbability: -960.518716192462
Current temperature: 8.5
Current logProbability: -

In [381]:
results

([6, 7, 8, 9, 10, 18, 17, 19, 14, 11, 12, 13, 5, 0, 15, 16, 1, 4, 3, 2],
 -875.8589138985831)